In [ ]:
import requests 
import datetime
import time
import hashlib
import hmac
from decimal import Decimal
import os


def truncate_number(number, desired_length):
    x = desired_length
    truncated_number = int(number * 10**x) / 10**x
    return truncated_number

def calculate_vwap(prices):
    total_value = 0
    total_volume = 0

    for bid in bids:
        price = float(bid[0])
        volume = float(bid[1])
        total_value += price * volume
        total_volume += volume

    if total_volume == 0:
        return 0  # To avoid division by zero error

    vwap = total_value / total_volume
    return vwap



class BinanceUS:
    
    def __init__(self):
        self.api_key='YOUR_API_KEY_HERE'
        self.private_key='YOUR_PRIVATE_API_KEY_HERE'
        self.exchange_info = self.get_exchange_info()
        self.symbols = None
        self.symbols_with_slash = None
        self.pairs = self.get_pairs()
        self.tick_size = self.set_tick_sizes()
        self.step_size = self.set_step_sizes()
        self.balances = None
        self.fees = self.get_fees()

        
    def get_account_data(self):
        
        private_key = self.private_key
        timestamp = str(int(time.time() * 1000))
        query_string = f"timestamp={timestamp}"
        signature = hmac.new(private_key.encode(), query_string.encode(), hashlib.sha256).hexdigest()
        url = f"https://api.binance.us/api/v3/account?{query_string}&signature={signature}"
        headers = {"X-MBX-APIKEY": self.api_key}

        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            account_data = response.json()
            return account_data
        else:
            print(f"Error: {response.status_code}, {response.text}")
            return None


    def get_fees(self):
        account_data = self.get_account_data()
        maker_fee = account_data['commissionRates']['maker']
        taker_fee = account_data['commissionRates']['taker']
        self.balances = account_data['balances']
        return {'maker_fee': maker_fee, 'taker_fee': taker_fee}
    
    
    def get_balance(self, asset):
        account_data = self.get_account_data()
        
        self.balances = account_data['balances']
        if self.balances != None:
            for balance in self.balances:
                if balance['asset'] == asset:
                    asset_balance = balance['asset']
                    free_balance = float(balance['free'])
                    locked_balance = float(balance['locked'])
                    return {'asset': asset_balance, 'free': free_balance, 'locked': locked_balance}
            return None, None
    
    def get_minimum_trade_size(self, symbol):
        pairs_info = self.pairs
        min_qty = None  # Initialize min_qty variable
        
        if symbol in pairs_info:
            min_qty = float(pairs_info[symbol].get('min_qty', None))
        
        return min_qty

    def get_minimum_notional(self, symbol):
        pairs_info = self.pairs
        min_notional = None
        if symbol in pairs_info:
            min_notional = float(pairs_info[symbol].get('min_notional', None))  
            return min_notional    
    
    def get_exchange_info(self):
        url = 'https://api.binance.us/api/v3/exchangeInfo'
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            self.exchange_info = data
            return self.exchange_info
        
    def get_pairs(self):
        pairs_info = {}
        syms = []
        syms_w_slash = {}
        
        for symbol in self.exchange_info['symbols']:
            status = symbol['status']
            base_asset = symbol['baseAsset']
            quote_asset = symbol['quoteAsset']
            trading_pair = f'{base_asset}{quote_asset}'
            sym = base_asset + quote_asset
            sym_w_slash = base_asset + '/' + quote_asset

            if status == 'TRADING':
                syms.append(sym)
                syms_w_slash[sym] = sym_w_slash
                tick_size = None
                step_size = None
                min_qty = None
                base_asset_precision = symbol['baseAssetPrecision']
                quote_asset_precision = symbol['quoteAssetPrecision']
                base_commission_precision = symbol['baseCommissionPrecision']
                quote_commission_precision = symbol['quoteCommissionPrecision']

                for filter_info in symbol['filters']:
                    if filter_info['filterType'] == 'PRICE_FILTER':
                        tick_size = filter_info['tickSize']
                    elif filter_info['filterType'] == 'LOT_SIZE':
                        step_size = filter_info['stepSize']
                        min_qty = filter_info['minQty']
                    elif filter_info['filterType'] == 'MIN_NOTIONAL':
                        min_notional = filter_info['minNotional']

                pairs_info[trading_pair] = {
                    'symbol': f'{base_asset}/{quote_asset}', 
                    'base': base_asset, 
                    'quote': quote_asset,
                    'tick_size': tick_size,
                    'step_size': step_size,
                    'min_qty': min_qty,
                    'min_notional': min_notional,
                    'base_asset_precision': base_asset_precision,
                    'quote_asset_precision': quote_asset_precision,
                    'base_commission_precision': base_commission_precision,
                    'quote_commission_precision': quote_commission_precision
                }
        self.symbols = sorted(syms)
        self.symbols_with_slash = syms_w_slash
        return pairs_info
    
        
    def set_step_sizes(self):
        step_sizes = {}
        for symbol, info in self.pairs.items():
            step_sizes[symbol] = float(info['step_size'])
        return step_sizes
   

    def get_step_size(self, symbol):
        return self.step_size.get(symbol, None)
    
    
    def set_tick_sizes(self):
        tick_sizes = {}
        for symbol, info in self.pairs.items():
            tick_sizes[symbol] = float(info['tick_size'])
        return tick_sizes
  
    def get_tick_size(self, symbol):
        return self.tick_size.get(symbol, None)

    def decimal_length(self, number):
        decimal_number = Decimal(str(number))
        _, num_decimals = decimal_number.as_tuple()[1:3]
        return max(-num_decimals, 0)
    
    def get_tick_length(self, symbol):
        tick_size = self.get_tick_size(symbol)
        tick_length = self.decimal_length(tick_size)
        return tick_length        

    def send_ioc_order(self, side, symbol, quantity, price):
        try:

            tick_length = self.get_tick_length(symbol)
            min_notional = self.get_minimum_notional(symbol)
            step_size = self.get_step_size(symbol)
            step_size_length = self.decimal_length(step_size)
            
            _quantity = truncate_number(quantity, step_size_length)

            _price = truncate_number(price, tick_length)
            
            _price = "{:.{}f}".format(price, tick_length)
            
            api_key = self.api_key
            secret_key = self.private_key
            endpoint = 'https://api.binance.us/api/v3/order'
            timestamp = int(time.time() * 1000)
            payload = {
                'symbol': symbol.upper(),
                'side': side.upper(),
                'type': 'LIMIT',
                'timeInForce': 'IOC',
                'quantity': str(_quantity),
                'price': _price,
                'timestamp': timestamp,
                'recvWindow': 5000,
            }
            print(f'payload: {payload}')
            query_string = '&'.join([f'{k}={v}' for k, v in payload.items()])
            signature = hmac.new(secret_key.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()
            payload['signature'] = signature
            headers = {'X-MBX-APIKEY': api_key, }
            print(f'    -> tick_length: {tick_length} min_notional: {min_notional} step_size: {step_size}')
            response = requests.post(endpoint, headers=headers, params=payload)
            response_data = response.json()
            return response_data


        except Exception as e:
            print("Error occurred in send_fok_order:", e)
    
            
    def get_trade_data(self, symbol, limit=None):
        if limit == None: limit = 30
        if limit > 1000: limit = 1000
        url = f"https://api.binance.us/api/v3/trades?symbol={symbol}&limit={limit}"
        response = requests.get(url)
        if response.status_code == 200:
            trade_data = response.json()
            return trade_data

    def analyze_historical_trades(self, symbol, limit=None):
        
        trade_data = self.get_trade_data(symbol, limit)

        net_volume = []
        bid_hit_count = 0
        offer_lifted_count = 0

        tmp_time = []
        total_len = len(trade_data)
        
        bid_hit_volume = []
        offer_lifted_volume = []

        for i in trade_data:

            price = float(i['price'])
            qty = float(i['qty'])
            quote_qty = float(i['quoteQty'])
            time = int(i['time'])    
            tmp_time.append(time)
            is_buyer_maker = i['isBuyerMaker']

            if is_buyer_maker:
                bid_hit_count += 1
                net_volume.append(-qty)
                bid_hit_volume.append(-qty)

            elif not is_buyer_maker:
                offer_lifted_count += 1
                net_volume.append(qty)
                offer_lifted_volume.append(qty)

        min_time = min(tmp_time)
        max_time = max(tmp_time)
        
        slope = trend_direction(net_volume)
        
        net_volume_sum = sum(net_volume)
        
        direction = None
        
        if slope > 0:
            direction = 'buy'
            
        elif slope < 0:
            direction = 'sell'

        
        obj = {
            'symbol': symbol,
            'length': total_len,
            'start_time': min_time,
            'end_time': max_time,
            'duration_seconds': (max_time-min_time)/1000,
            'bids_hit': {'count': bid_hit_count},
            'offers_lifted': {'count': offer_lifted_count},
            'bid_hit_volume': bid_hit_volume,
            'offer_lifted_volume': offer_lifted_volume,
            'slope': slope,
            'net_volume_sum': net_volume_sum,
            'direction': direction
        }

        return obj





    def get_order_book_depth(self, symbol):
        url = f"https://api.binance.us/api/v3/depth?symbol={symbol}"
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        
    
    def analyze_order_book_depth(self, symbol):
        
        tick_length = self.get_tick_length(symbol)
        
        order_book_depth = self.get_order_book_depth(symbol)
        
        bids = order_book_depth['bids']
        asks = order_book_depth['asks']

        best_bid = float(order_book_depth['bids'][0][0])
        best_bid_qty = float(order_book_depth['bids'][0][1])

        best_ask = float(order_book_depth['asks'][0][0])
        best_ask_qty = float(order_book_depth['asks'][0][1])

        mid_price = float(round((best_ask + best_bid)/2, tick_length + 1))
        
        spread = round(best_ask - best_bid, tick_length)
        bid_vwap = float(round(calculate_vwap(bids), tick_length + 1))
        ask_vwap = float(round(calculate_vwap(asks), tick_length + 1))
        
        mid_vwap = float(round((ask_vwap + bid_vwap)/2, tick_length + 1))
        
        direction = None
        
        if mid_price > mid_vwap:
            direction = 'sell'
        if mid_vwap > mid_price:
            direction = 'buy'
        
        
        obj ={
            'symbol': symbol,
            'best_bid_qty': best_bid_qty,
            'best_bid': best_bid,
            'mid_price': mid_price,
            'best_ask': best_ask,
            'best_ask_qty': best_ask_qty,
            'spread': spread,
            'bid_vwap': bid_vwap,
            'mid_vwap': mid_vwap,
            'ask_vwap': ask_vwap,
            'direction': direction
        }

        return obj
    
    
    def get_order(self, orderId, symbol):
        timestamp = str(int(time.time() * 1000))
        api_url = "https://api.binance.us/api/v3/order"
        query_string = f"orderId={orderId}&symbol={symbol}&timestamp={timestamp}"
        signature = hmac.new(self.private_key.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()
        headers = {"X-MBX-APIKEY": self.api_key}
        params = {
            "orderId": orderId,
            "symbol": symbol,
            "timestamp": timestamp,
            "signature": signature
        }

        response = requests.get(api_url, params=params, headers=headers)
        return response.json()

    
    def cancel_order(self, orderId, symbol):
        timestamp = str(int(time.time() * 1000))
        api_url = "https://api.binance.us/api/v3/order"
        query_string = f"orderId={orderId}&symbol={symbol}&timestamp={timestamp}"
        signature = hmac.new(self.private_key.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

        headers = {"X-MBX-APIKEY": self.api_key}

        params = {
            "orderId": orderId,
            "symbol": symbol,
            "timestamp": timestamp,
            "signature": signature
        }

        response = requests.delete(api_url, params=params, headers=headers)
        return response.json()

    


b = BinanceUS()
black_list_pairs = ['WBTCBTC'] # Pairs you do not want to trade, may be a stablecoin or low volume pairs, etc
white_list_pairs = ['BTCUSDC'] # No trading commision on Binance US as of Feb 01, 2024 *** may change - double check first!
tradeable_symbols = b.symbols


for symbol in white_list_pairs:
    
    if symbol not in black_list_pairs:

        base, quote = b.symbols_with_slash.get(symbol).split('/')
        base_balance = b.get_balance(base)['free']
        quote_balance = b.get_balance(quote)['free']

        min_notional = b.get_minimum_notional(symbol)
        min_notional_length = b.decimal_length(min_notional)
        min_tick_size = b.get_tick_size(symbol)
        tick_length = b.get_tick_length(symbol)
        step_size = b.get_step_size(symbol)
        step_size_length = b.decimal_length(step_size)

        hist_trades = b.analyze_historical_trades(symbol, 30)
        ht_direction = hist_trades['direction']

        order_book_data = b.analyze_order_book_depth(symbol)
        ob_direction = order_book_data['direction']

        best_bid = order_book_data['best_bid']
        best_ask = order_book_data['best_ask']
        mid_price = order_book_data['mid_price']
        spread = order_book_data['spread']
        mid_vwap = order_book_data['mid_vwap']


        if mid_vwap > mid_price:

            min_quantity = 0.00003 # Set your minimum quantity 
            buy_order_repsonse = b.send_ioc_order('buy', symbol, min_quantity, best_ask)

